In [1]:
!ls

Client_Project_Ixchel.ipynb
Client_Project_John.ipynb
Client_Project_Kevin.ipynb
Ixchel_Twitter.ipynb
NotifyNYC_tweets.csv
Weather
converting-text-files-to-combined-dictionary.ipynb
data
mycsv.csv
raw_tweets
saved_tweets.csv
twitter_credentials.json
working_df.csv


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./raw_tweets/raw_tweets.csv')

In [4]:
df.drop(columns='Unnamed: 0', inplace=True)

In [5]:
df.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
0,267716394043465728,2012-11-11 14:52:16,NaN,False,False,NaN,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy
1,298660085255770112,2013-02-05 00:11:27,NaN,False,False,NaN,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY
2,21860028236,2010-08-22 17:33:44,NaN,False,False,NaN,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1
3,1069021302667833344,2018-12-01 19:11:54,NaN,False,False,NaN,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale
4,767077182124486656,2016-08-20 15:13:46,NaN,False,False,NaN,3.0,0.0,7.0,.@ConEdison is responding to a power outag...,/NotifyNYC/status/767077182124486656,16145875.0,NotifyNYC


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4375 entries, 0 to 4374
Data columns (total 13 columns):
ID               4375 non-null object
datetime         4375 non-null object
has_media        150 non-null object
is_reply         4374 non-null object
is_retweet       4374 non-null object
medias           149 non-null object
nbr_favorite     4374 non-null float64
nbr_reply        4374 non-null float64
nbr_retweet      4374 non-null float64
text             4374 non-null object
url              4373 non-null object
user_id          4373 non-null float64
usernameTweet    4373 non-null object
dtypes: float64(4), object(9)
memory usage: 444.4+ KB


In [7]:
# check datatypes
df.dtypes

ID                object
datetime          object
has_media         object
is_reply          object
is_retweet        object
medias            object
nbr_favorite     float64
nbr_reply        float64
nbr_retweet      float64
text              object
url               object
user_id          float64
usernameTweet     object
dtype: object

In [8]:
# converting the datetime column to datetime format
# won't work because there there not all of the cells are formatted correctly

#pd.to_datetime(df['datetime']).head()

# returns 'ValueError: month must be in 1..12'

In [9]:
# locate the datetime value that is not in the correct format
for i in df.datetime:
    if i[5:7] not in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        print(i)

20567184


In [10]:
# locate the index number of that value
df[df.datetime == '20567184']

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
2808,/Maggie4Obama/status/108170030474203136,20567184,Maggie4Obama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# inspect the item
df.iloc[2808]

ID               /Maggie4Obama/status/108170030474203136
datetime                                        20567184
has_media                                   Maggie4Obama
is_reply                                             NaN
is_retweet                                           NaN
medias                                               NaN
nbr_favorite                                         NaN
nbr_reply                                            NaN
nbr_retweet                                          NaN
text                                                 NaN
url                                                  NaN
user_id                                              NaN
usernameTweet                                        NaN
Name: 2808, dtype: object

In [12]:
# drop that row since it is mostly bad data
df.drop(index=2808, inplace=True)

In [13]:
# try converting the 'datetime' column to datetime again
df.datetime = pd.to_datetime(df.datetime)

In [14]:
# check that it worked
df.dtypes

ID                       object
datetime         datetime64[ns]
has_media                object
is_reply                 object
is_retweet               object
medias                   object
nbr_favorite            float64
nbr_reply               float64
nbr_retweet             float64
text                     object
url                      object
user_id                 float64
usernameTweet            object
dtype: object

In [15]:
# check nulls
df.isnull().sum()

ID                  0
datetime            0
has_media        4225
is_reply            0
is_retweet          0
medias           4225
nbr_favorite        0
nbr_reply           0
nbr_retweet         0
text                0
url                 1
user_id             1
usernameTweet       1
dtype: int64

In [16]:
# impute 'no media' for the null media columns
df['has_media'] = df.has_media.fillna('no media')
df['medias'] = df.has_media.fillna('no media')
df.isnull().sum()

ID               0
datetime         0
has_media        0
is_reply         0
is_retweet       0
medias           0
nbr_favorite     0
nbr_reply        0
nbr_retweet      0
text             0
url              1
user_id          1
usernameTweet    1
dtype: int64

In [30]:
#check out the null url
df[df.url.isnull()]

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
2807,108170030474203136,2011-08-29 09:31:59,no media,False,False,no media,0.0,0.0,0.0,TomSmolka Tom Smolka,NaN,NaN,NaN


In [31]:
# drop that row since it is mostly bad data
df.drop(index=2807, inplace=True)

In [33]:
df.isnull().sum()

ID               0
datetime         0
has_media        0
is_reply         0
is_retweet       0
medias           0
nbr_favorite     0
nbr_reply        0
nbr_retweet      0
text             0
url              0
user_id          0
usernameTweet    0
dtype: int64

#### Unique Values

In [17]:
unique = []
for i in df.columns:
    uniques = {}
    uniques['column'] = i 
    uniques['unique values'] = df[i].nunique()
    unique.append(uniques)
pd.DataFrame(unique)

,column,unique values
0,ID,4374
1,datetime,4254
2,has_media,2
3,is_reply,2
4,is_retweet,1
5,medias,2
6,nbr_favorite,33
7,nbr_reply,19
8,nbr_retweet,45
9,text,4116


#### Consider The Most Common Twitter Handles
There are many handles that have multiple tweets. These include official handles and news organizations.

We can consider using some of these as target indicators, particularly those that are official handles that are sharing information the utility would already know about.  

We will also want to consider extracting these from the dataframe before parsing for language, since they're likely to bias any model looking for common language (i.e. many of the tweets from NotifyNYC follow a format along the lines of "@Conedison is responding to a power outage in {zip-code}).

In [18]:
df.usernameTweet.value_counts()[:20]

NotifyNYC         641
EastCoast911IA    135
ConEdison         113
Shield1631         95
outagetracker      31
NYCityAlerts       20
WHCREBT            16
BNNnews            14
NYGovCuomo         14
StatenIslUSA       14
BreakingNewsNY     13
nyc311             13
res7cuefox5        13
BreakingNewsUS     13
bnndesk            12
yonkersoem40       11
TalkoftheSound     11
fox5ny             11
AlwaysActions      11
JoshuaRockoff      10
Name: usernameTweet, dtype: int64

In [19]:
df[df.usernameTweet == 'NotifyNYC'].head(1)

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
4,767077182124486656,2016-08-20 15:13:46,no media,False,False,no media,3.0,0.0,7.0,.@ConEdison is responding to a power outag...,/NotifyNYC/status/767077182124486656,16145875.0,NotifyNYC


In [20]:
df[df.usernameTweet == 'EastCoast911IA'][:1]

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
7,764563820354408448,2016-08-13 16:46:34,no media,False,False,no media,0.0,0.0,0.0,NotifyNYC: .ConEdison is responding to a powe...,/EastCoast911IA/status/764563820354408448,2.778990e+09,EastCoast911IA


In [22]:
df[df.usernameTweet == 'Shield1631'].head(1)

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
122,997027507995140096,2018-05-17 04:14:16,no media,False,False,no media,0.0,0.0,0.0,.ConEdison is responding to a power outage ...,/Shield1631/status/997027507995140096,3.493897e+09,Shield1631


Looks like many (if not all) of the tweets from 'EastCoast911IA' and 'Shield1631' are just restating NotifyNYC tweets

In [21]:
df[df.usernameTweet == 'fox5ny'].head(1)

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
24,384683458410201088,2013-09-30 10:17:38,no media,False,False,no media,1.0,0.0,0.0,MT @NotifyNYC Power outage in Lighthouse ...,/fox5ny/status/384683458410201088,9229672.0,fox5ny


It turns out that many groups, particulatly news organizations, retweet @NotifyNYC. These are almost definitely duplicate tweets, so we want to exclude them from our analysis. Below we accomplish this using various masks to identify the retweets and systematically pair down the dataframe. 

In [23]:
# first, extract NotifyNYC so that we don't accidentally remove tweets from that handle
df_with_only_NotifyNYC = df[df.usernameTweet == 'NotifyNYC']

# save that to a csv
df_with_only_NotifyNYC.to_csv('NotifyNYC_tweets.csv')

# and create a dataframe without NotifyNYC that can be paired down
df_without_NotifyNYC = df[df.usernameTweet != 'NotifyNYC']

In [24]:
# create a mask to identify "retweets" of NotifyNYC
retweets_notify = df_without_NotifyNYC.text.str.contains('NotifyNYC')

# print the number of tweets in the mask
print(f'{len(df_without_NotifyNYC[retweets_notify])} retweets in the mask')
print('')

# save a new dataframe that excludes these tweets
df_without_notify_retweets = df_without_NotifyNYC[~retweets_notify]

# print out the top twitter handles after removing the retweets
print(df_without_notify_retweets.usernameTweet.value_counts()[:5])

361 retweets in the mask

ConEdison        113
Shield1631        95
outagetracker     28
NYCityAlerts      19
BNNnews           14
Name: usernameTweet, dtype: int64


In [25]:
# create a mask to identify "retweets" of NotifyNYC
# noting that every 'Shild1631' tweet in our dataframe was a NotifyNYC retweet
retweets_notify = df_without_NotifyNYC.usernameTweet == 'Shield1631'

# print the number of tweets in the mask
print(f'{len(df_without_notify_retweets[retweets_notify])} retweets in the mask')
print('')

# save a new dataframe that excludes these tweets
df_without_notify_retweets = df_without_notify_retweets[~retweets_notify]

# print out the top twitter handles after removing the retweets
print(df_without_notify_retweets.usernameTweet.value_counts()[:5])

95 retweets in the mask

ConEdison        113
outagetracker     28
NYCityAlerts      19
BNNnews           14
NYGovCuomo        14
Name: usernameTweet, dtype: int64


/Users/kevinc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/kevinc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


In [26]:
# create a mask to identify "retweets" of NotifyNYC
# taking advantage of the strange character spacing in some of the tweets
retweets_notify = df_without_notify_retweets.text.str.contains('is responding to a  p')

# print the number of tweets in the mask
print(f'{len(df_without_notify_retweets[retweets_notify])} retweets in the mask')
print('')

# save a new dataframe that excludes these tweets
df_without_notify_retweets = df_without_notify_retweets[~retweets_notify]

# print out the top twitter handles after removing the retweets
print(df_without_notify_retweets.usernameTweet.value_counts()[:5])

57 retweets in the mask

ConEdison        113
outagetracker     28
NYCityAlerts      19
NYGovCuomo        14
BNNnews           14
Name: usernameTweet, dtype: int64


In [39]:
df_without_notify_retweets

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet
0,267716394043465728,2012-11-11 14:52:16,no media,False,False,no media,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,5.042797e+08,RealJezzy
1,298660085255770112,2013-02-05 00:11:27,no media,False,False,no media,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,9.095948e+08,meirBGNY
2,21860028236,2010-08-22 17:33:44,no media,False,False,no media,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,6.888657e+07,olgushka1
3,1069021302667833344,2018-12-01 19:11:54,no media,False,False,no media,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,1.239352e+07,jimcasale
5,262590240374550528,2012-10-28 12:22:46,no media,False,False,no media,0.0,0.0,0.0,How to report a power outage (& you should...,/EastchesterEMS/status/262590240374550528,3.214100e+08,EastchesterEMS
6,969771441569845248,2018-03-02 22:08:23,no media,False,False,no media,1.0,1.0,0.0,@ ConEdison major power outage in Harriso...,/Beardowling/status/969771441569845248,3.048370e+08,Beardowling
9,263041619375321088,2012-10-29 18:16:23,no media,False,False,no media,0.0,0.0,0.0,Handy #Sandy Maps: ConEd power shutdown w...,/heatheremerrick/status/263041619375321088,7.352872e+06,heatheremerrick
10,263555684828852224,2012-10-31 04:19:06,no media,False,False,no media,0.0,0.0,0.0,Manhattan Power Outages to Continue for Da...,/BizInfoWoman/status/263555684828852224,1.448817e+08,BizInfoWoman
11,263392417477173248,2012-10-30 17:30:20,no media,False,False,no media,0.0,0.0,0.0,Helpful link: @ConEdison service outage map...,/the59thStBridge/status/263392417477173248,1.673217e+07,the59thStBridge
12,1014017018008399872,2018-07-03 01:24:31,no media,False,False,no media,1.0,0.0,0.0,im convinced that coned is lowering our pow...,/ExcaliburUmbra_/status/1014017018008399872,7.753880e+17,ExcaliburUmbra_


# Pre-Processing

### TweetTokenize and Lemmatize all of the Titles

In [44]:
# import tokenizer and stemmer
from nltk import word_tokenize, TweetTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [112]:
# define a function that takes in a tweet
# and tokenizes and stems that tweet
def stem_tweet(tweet):    
    stm = PorterStemmer()
    tokenized = word_tokenize(tweet)
    stem_tokes = []
    for toke in tokenized:
        stem_tokes.append(stm.stem(toke))
    stem_tokes

    comb = ''
    for stemmed in stem_tokes:
        comb += stemmed + ' '
    return comb

# define a function that takes in a tweet
# and tweet_tokenizes and lemmatizes that tweet
def lem_tweet(tweet):    
    lem = WordNetLemmatizer()
    tokenizer = TweetTokenizer()
    tokenized = tokenizer.tokenize(tweet)
    lemmed_tokes = []
    for toke in tokenized:
        lemmed_tokes.append(lem.lemmatize(toke))
    lemmed_tokes

    comb = ''
    for lemmed in lemmed_tokes:
        comb += lemmed + ' '
    return comb

In [113]:
# check that the function works
stem_tweet(df_without_notify_retweets.text[3])

"about that power outag that is now over ... from @ verizon `` the commerci power issu in your area may have been clear . If you still have issu with your verizon servic , click here . '' from @ conedison < cricket > "

In [47]:
# create a new column by applying the function to the title column
# and check out the head to see if it worked
df_without_notify_retweets['lemmed_tweet'] = df_without_notify_retweets.text.apply(lem_tweet)
df_without_notify_retweets.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet
0,267716394043465728,2012-11-11 14:52:16,no media,False,False,no media,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy,ConEd : NY Sandy power outage slip ; cost coul...
1,298660085255770112,2013-02-05 00:11:27,no media,False,False,no media,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY,@ ConEdison Power outage in queen
2,21860028236,2010-08-22 17:33:44,no media,False,False,no media,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1,Power outage in the Whitestone section of Quee...
3,1069021302667833344,2018-12-01 19:11:54,no media,False,False,no media,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale,about that power outage that is now over ... F...
5,262590240374550528,2012-10-28 12:22:46,no media,False,False,no media,0.0,0.0,0.0,How to report a power outage (& you should...,/EastchesterEMS/status/262590240374550528,321410001.0,EastchesterEMS,How to report a power outage ( & you should ! ...


In [48]:
# check out the tail, too
df_without_notify_retweets.tail()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet
4367,337934949967597571,2013-05-24 10:15:45,no media,False,False,no media,0.0,0.0,0.0,Anyone experiencing power loss on #Roosevelt...,/SenatorSerrano/status/337934949967597571,3.695740e+07,SenatorSerrano,Anyone experiencing power loss on #RooseveltIs...
4369,266333200966623232,2012-11-07 19:15:58,no media,False,False,no media,0.0,0.0,3.0,RT @kmbtweets : RT @SINYCliving : Power out...,/SINYCliving/status/266333200966623232,1.524081e+08,SINYCliving,RT @kmbtweets : RT @SINYCliving : Power outage...
4371,264374015714553858,2012-11-02 10:30:51,no media,True,False,no media,0.0,0.0,1.0,@ hhertzof Please check out our outage map ...,/ConEdison/status/264374015714553858,2.026208e+07,ConEdison,@ hhertzof Please check out our outage map at ...
4372,308557748671479808,2013-03-04 07:41:14,no media,False,False,no media,0.0,1.0,0.0,@ ConEdison Partial power outage in Green...,/curlycakes/status/308557748671479808,2.760942e+08,curlycakes,@ ConEdison Partial power outage in Greenwood ...
4373,969704423621648384,2018-03-02 17:42:05,no media,False,False,no media,0.0,1.0,0.0,"Just out of curiosity @EversourceCT , will the...",/yelenadasher/status/969704423621648384,2.791989e+09,yelenadasher,"Just out of curiosity @EversourceCT , will the..."


In [49]:
#df_without_notify_retweets.to_csv('working_df_without_retweets.csv')

## Model Prep

### CountVectorize from Unlemmed

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [52]:
# Instantiate
cv = CountVectorizer(stop_words='english', 
                     ngram_range=(1,1)
                    )

In [53]:
# Fit and Transform
sparse_df_unlemmed = cv.fit_transform(df_without_notify_retweets.text)

In [54]:
dense_df_unlemmed = pd.DataFrame(sparse_df_unlemmed.todense(), columns=cv.get_feature_names())
dense_df_unlemmed.head()

,00,000,0004cd37823361e17f570,0025,004681,0075,008,00am,00pm,01,...,zq1l1q,zq77qw,zqccse,zrhytqylft,zup9nhrybuektc1lcewgbpjqjpebeptl,zup9nhrybuhm1ks2tvqbgrdi67_ocwsu,zuqbnywg1dl,zxutv,zzwxbkvgxy,うちはまだ停電してないだけましらしい
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [55]:
len(cv.get_feature_names())

6777

### CountVectorize from Lemmed

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

In [57]:
# Instantiate
cv = CountVectorizer(stop_words='english', 
                     ngram_range=(1,1)
                    )

In [58]:
# Fit and Transform
sparse_df_lemmed = cv.fit_transform(df_without_notify_retweets.lemmed_tweet)

In [59]:
dense_df_lemmed = pd.DataFrame(sparse_df_lemmed.todense(), columns=cv.get_feature_names())
dense_df_lemmed.head()

,00,000,0004,001,002092,0025,004681,0075,008,00am,...,zpzuap,zq1l1q,zq77qw,zqccse,zrhytqylft,zup,zuqbnywg1dl,zxutv,zzwxbkvgxy,うちはまだ停電してないだけましらしい
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [60]:
len(cv.get_feature_names())

6648

In [61]:
cv.get_feature_names()[6000:6008]

['tonight',
 'tonights',
 'toogooden16',
 'took',
 'tool',
 'toolong',
 'torch',
 'tornados']

## Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
km = KMeans(n_clusters=3, random_state=42)

In [ ]:
X_sc = ss.fit_transform(dense_df_lemmed)
km.fit(X_sc)

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
silhouette_score(X_sc, km.labels_)

In [ ]:
len(df[km.labels_ == 0])

In [ ]:
len(df[km.labels_ == 1])

In [ ]:
len(df[km.labels_ == 2])

# Spacy Processing

In [62]:
import spacy
nlp = spacy.load('en')
doc1 = nlp(df_without_notify_retweets.lemmed_tweet[2])
print(doc1)

Power outage in the Whitestone section of Queens . To report a loss of electric service , contact Con Edison at 1-800- 75 - CONED 


## Part-of-Speech Tagging

In [63]:
# check out what this can do

for token in doc1:
    print('{} - {}'.format(token, token.pos_))

Power - NOUN
outage - NOUN
in - ADP
the - DET
Whitestone - PROPN
section - NOUN
of - ADP
Queens - PROPN
. - PUNCT
To - PART
report - VERB
a - DET
loss - NOUN
of - ADP
electric - ADJ
service - NOUN
, - PUNCT
contact - NOUN
Con - PROPN
Edison - PROPN
at - ADP
1 - NUM
- - PUNCT
800- - NUM
75 - NUM
- - PUNCT
CONED - NOUN


#### Extract Nouns from Tweets

In [64]:
# define a function to extract a tweet's nouns 
# and return them in a single string,
# so that the string can then be added back into the dataframe

def extract_NOUN(tweet):
    doc = nlp(tweet)
    NOUNS = []
    for token in doc:
        if token.pos_ == 'NOUN':
            NOUNS.append(str(token))   
    #print(NOUNS)
    comb = ''
    for token in NOUNS:
        comb += token + ' '
    return comb

In [66]:
# test out the function

extract_NOUN(df_without_notify_retweets.lemmed_tweet[0])

'ConEd power outage slip cost http '

In [68]:
# apply the function to every lemmed tweet to create a new 'nouns' column

df_without_notify_retweets['nouns'] = df_without_notify_retweets.lemmed_tweet.apply(extract_NOUN)
df_without_notify_retweets.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet,nouns
0,267716394043465728,2012-11-11 14:52:16,no media,False,False,no media,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy,ConEd : NY Sandy power outage slip ; cost coul...,ConEd power outage slip cost http
1,298660085255770112,2013-02-05 00:11:27,no media,False,False,no media,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY,@ ConEdison Power outage in queen,outage queen
2,21860028236,2010-08-22 17:33:44,no media,False,False,no media,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1,Power outage in the Whitestone section of Quee...,Power outage section loss service contact CONED
3,1069021302667833344,2018-12-01 19:11:54,no media,False,False,no media,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale,about that power outage that is now over ... F...,power outage @Verizon power issue area issue s...
5,262590240374550528,2012-10-28 12:22:46,no media,False,False,no media,0.0,0.0,0.0,How to report a power outage (& you should...,/EastchesterEMS/status/262590240374550528,321410001.0,EastchesterEMS,How to report a power outage ( & you should ! ...,power outage http://www bckey bclid bctid … ou...


#### Extract Proper Nouns from Tweets

In [69]:
# define a function to extract a tweet's proper nouns 
# and return them in a single string,
# so that the string can then be added back into the dataframe

def extract_PROPN(tweet):
    doc = nlp(tweet)
    PROPN = []
    for token in doc:
        if token.pos_ == 'PROPN':
            PROPN.append(str(token))   
    #print(PROPN)
    comb = ''
    for token in PROPN:
        comb += token + ' '
    return comb

In [73]:
# test the function
extract_PROPN(df_without_notify_retweets.lemmed_tweet[0])

'NY Sandy dlvr.it/2Sss2H '

In [74]:
# apply the function to every lemmed tweet to create a new 'proper_nouns' column
df_without_notify_retweets['proper_nouns'] = df_without_notify_retweets.lemmed_tweet.apply(extract_PROPN)
df_without_notify_retweets.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet,nouns,proper_nouns
0,267716394043465728,2012-11-11 14:52:16,no media,False,False,no media,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy,ConEd : NY Sandy power outage slip ; cost coul...,ConEd power outage slip cost http,NY Sandy dlvr.it/2Sss2H
1,298660085255770112,2013-02-05 00:11:27,no media,False,False,no media,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY,@ ConEdison Power outage in queen,outage queen,ConEdison Power
2,21860028236,2010-08-22 17:33:44,no media,False,False,no media,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1,Power outage in the Whitestone section of Quee...,Power outage section loss service contact CONED,Whitestone Queens Con Edison
3,1069021302667833344,2018-12-01 19:11:54,no media,False,False,no media,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale,about that power outage that is now over ... F...,power outage @Verizon power issue area issue s...,Verizon @ConEdison
5,262590240374550528,2012-10-28 12:22:46,no media,False,False,no media,0.0,0.0,0.0,How to report a power outage (& you should...,/EastchesterEMS/status/262590240374550528,321410001.0,EastchesterEMS,How to report a power outage ( & you should ! ...,power outage http://www bckey bclid bctid … ou...,ConEd AQ AAAA 3ijeYFE ~ Zup 9nhRYBuHm1KS2tVqBG...


## Noun-Phrase-Chunking

In [75]:
for chunk in doc1.noun_chunks:
    print('{} - {}'.format(chunk, chunk.label_))

Power outage - NP
the Whitestone section - NP
Queens - NP
a loss - NP
electric service - NP
- CONED - NP


#### Extract Noun Phrase Chunks from Tweets

In [77]:
for tweet in df_without_notify_retweets.lemmed_tweet[:30]:
    #print(nlp(i))
    doc = nlp(tweet)
    NP = []
    for chunk in doc.noun_chunks:
        NP.append(str(chunk))   
    print(NP)

['NY Sandy power outage slip', 'cost', '/ dlvr.it/2Sss2H']
['ConEdison Power outage', 'queen']
['Power outage', 'the Whitestone section', 'Queens', 'a loss', 'electric service', '- CONED']
['that power outage', '@Verizon', 'The commercial power issue', 'your area', 'you', 'issue', 'your Verizon service', '@ConEdison']
['a power outage', 'you', 'ConEd', 'bckey', 'AQ', 'bctid', '| Live ConEd outage map', 'reference', '/ apps.coned.com/stormcenter_ex ternal / default.html …']
['ConEdison major power outage', 'Harrison NY']
['Sandy Maps', 'ConEd power shutdown warning', '0.0004 cd37823361e17f570', 'ie', 'UTF', '… Current ConEd outage', '/ apps.coned.com/stormcenter_ex ternal / default.html …']
['Manhattan Power Outages', 'Days', 'ConEd']
['Helpful link', '@ConEdison service outage map', '/ apps.coned.com/stormcenter_ex ternal / default.html … Power', 'estimated restoration time', 'Manhattan']
['i', 'our power', 'no outage', 'you', 'your infrastructure']
['# ConEd', '18,200 power outage', '

## Named-Entity-Recognition

In [78]:
for ent in doc1.ents:
    print(ent.text, '-', ent.label_)

Whitestone - GPE
Queens - ORG
Con Edison - PERSON
1 - CARDINAL


#### Create a new column that contains Geopolitical Entities from Tweets

In [79]:
# define a function that takes in a tweet and returns a single string
# that contains only the Geopolitical Entities from that tweet
def extract_GPE(tweet):
    doc = nlp(tweet)
    GPE = []
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            GPE.append(str(ent.text))   
    #print(GPE)
    comb = ''
    for token in GPE:
        comb += token + ' '
    return comb

In [81]:
# check that the function works
extract_GPE(df_without_notify_retweets.lemmed_tweet[0])

''

In [82]:
# apply the function to every lemmed tweet to create a new 'geopolitical_ents' column
df_without_notify_retweets['geopolitical_ents'] = df_without_notify_retweets.lemmed_tweet.apply(extract_GPE)
df_without_notify_retweets.head()

,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet,nouns,proper_nouns,geopolitical_ents
0,267716394043465728,2012-11-11 14:52:16,no media,False,False,no media,0.0,0.0,0.0,ConEd : NY Sandy power outages slip; costs...,/RealJezzy/status/267716394043465728,504279674.0,RealJezzy,ConEd : NY Sandy power outage slip ; cost coul...,ConEd power outage slip cost http,NY Sandy dlvr.it/2Sss2H,
1,298660085255770112,2013-02-05 00:11:27,no media,False,False,no media,0.0,0.0,0.0,@ ConEdison \n Power outage in queens,/meirBGNY/status/298660085255770112,909594764.0,meirBGNY,@ ConEdison Power outage in queen,outage queen,ConEdison Power,
2,21860028236,2010-08-22 17:33:44,no media,False,False,no media,0.0,0.0,0.0,Power outage in the Whitestone section of Q...,/olgushka1/status/21860028236,68886571.0,olgushka1,Power outage in the Whitestone section of Quee...,Power outage section loss service contact CONED,Whitestone Queens Con Edison,Whitestone
3,1069021302667833344,2018-12-01 19:11:54,no media,False,False,no media,0.0,1.0,0.0,about that power outage that is now over.....,/jimcasale/status/1069021302667833344,12393522.0,jimcasale,about that power outage that is now over ... F...,power outage @Verizon power issue area issue s...,Verizon @ConEdison,Verizon
5,262590240374550528,2012-10-28 12:22:46,no media,False,False,no media,0.0,0.0,0.0,How to report a power outage (& you should...,/EastchesterEMS/status/262590240374550528,321410001.0,EastchesterEMS,How to report a power outage ( & you should ! ...,power outage http://www bckey bclid bctid … ou...,ConEd AQ AAAA 3ijeYFE ~ Zup 9nhRYBuHm1KS2tVqBG...,


### Dependency Parsing
Can we use this to get items like "Lighthouse Hill" that aren't recognized by Named-Entity-Recognition?

In [ ]:
#df.lemmed_tweet[24]

In [ ]:
# from spacy import displacy

# nlp = spacy.load("en_core_web_sm")
# doc = nlp(df.lemmed_tweet[24])
# displacy.render(doc, style='dep')

In [83]:
df_without_notify_retweets.shape

(3220, 17)

In [95]:
df_temp = df_without_notify_retweets.sort_values('datetime')
df_temp = df_temp.reset_index()
df_temp

,index,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet,nouns,proper_nouns,geopolitical_ents
0,2597,159679592,2007-07-20 11:47:44,no media,False,False,no media,0.0,0.0,0.0,upset about power outage at Jango. ConEd ...,/mattiasstanghed/status/159679592,5.973102e+06,mattiasstanghed,upset about power outage at Jango . ConEd need...,power outage ConEd act,Jango,Jango
1,3010,821851312,2008-05-28 12:54:52,no media,True,False,no media,0.0,0.0,0.0,@ JonBurg We don't applaud ConEd for restor...,/awolk/status/821851312,8.297532e+06,awolk,@ JonBurg We don't applaud ConEd for restoring...,power outage point,JonBurg ConEd,
2,3328,831579275,2008-06-10 15:24:02,no media,False,False,no media,0.0,0.0,0.0,Voice mail from ConEd says we're close to a ...,/schlchtmyr/status/831579275,1.466381e+06,schlchtmyr,Voice mail from ConEd say we're close to a pow...,Voice mail power outage guy jackhammer,ConEd AC,
3,4202,864377956,2008-07-21 14:34:01,no media,False,False,no media,0.0,0.0,0.0,Neighborhood power outage . Goddamn ConEd ...,/corneilius/status/864377956,5.686132e+06,corneilius,Neighborhood power outage . Goddamn ConEd . Th...,Neighborhood power outage rivington st,Goddamn ConEd,
4,3811,1395303668,2009-03-26 12:29:56,no media,False,False,no media,0.0,0.0,0.0,Headed home early -- power outage at the o...,/deadlytango/status/1395303668,1.465168e+07,deadlytango,Headed home early - - power outage at the offi...,home power outage office Thanks,ConEd,
5,3573,1960011711,2009-05-29 10:16:30,no media,False,False,no media,0.0,0.0,0.0,@ wnyc has been having power outages this...,/macsamurai/status/1960011711,4.324300e+04,macsamurai,@ wnyc ha been having power outage this mornin...,wnyc power outage morning pledge week pledge,ConEd,
6,4281,2422668339,2009-07-01 12:55:33,no media,False,False,no media,0.0,0.0,0.0,Now that I'm having people work in my home mor...,/brianalvey/status/2422668339,4.119021e+06,brianalvey,Now that I'm having people work in my home mor...,people home outage power pm Losers,ConEd,
7,159,4484429286,2009-09-29 20:27:45,no media,False,False,no media,0.0,0.0,0.0,"Hates power outages , ConEd . Thinking abo...",/keledy/status/4484429286,9.550242e+06,keledy,"Hates power outage , ConEd . Thinking about us...",Hates power outage car alarm power source,ConEd,
8,2172,4893673114,2009-10-15 13:24:09,no media,False,False,no media,0.0,0.0,0.0,RT @NYCOEM : ConEd responding to a power ...,/bobtheastorian/status/4893673114,2.550778e+07,bobtheastorian,RT @NYCOEM : ConEd responding to a power outag...,@NYCOEM ConEd power outage neighborhood,RT Marine Park Brooklyn,Brooklyn
9,482,7068199124,2009-12-26 14:26:22,no media,False,False,no media,1.0,0.0,1.0,Show almost cancelled due to actual power o...,/Jay_A_Johnson/status/7068199124,9.486353e+07,Jay_A_Johnson,Show almost cancelled due to actual power outa...,power outage LOVE,Show ConEd,


In [105]:
df_2012_2017 = df_temp[(df_temp.datetime >= '2012') &  (df_temp.datetime <= '2018')]

In [117]:
# https://stackoverflow.com/questions/39805961/pandas-remove-seconds-from-datetime-index
df_2012_2017['datetime'] = df_2012_2017['datetime'].map(lambda x: x.replace(minute=0, second=0))
df_2012_2017.to_csv('data/nlp_tweets.csv')

/Users/kevinc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [115]:
df_2012_2017[df_2012_2017.proper_nouns.str.contains('AAAA')]

,index,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet,nouns,proper_nouns,geopolitical_ents
582,5,262590240374550528,2012-10-28 12:00:00,no media,False,False,no media,0.0,0.0,0.0,How to report a power outage (& you should...,/EastchesterEMS/status/262590240374550528,321410001.0,EastchesterEMS,How to report a power outage ( & you should ! ...,power outage http://www bckey bclid bctid … ou...,ConEd AQ AAAA 3ijeYFE ~ Zup 9nhRYBuHm1KS2tVqBG...,
653,1278,262985394629521408,2012-10-29 14:00:00,no media,False,False,no media,0.0,0.0,0.0,How to report a power outage (& you should...,/EastchesterEMS/status/262985394629521408,321410001.0,EastchesterEMS,How to report a power outage ( & you should ! ...,power outage http://www bckey bclid bctid … ou...,ConEd AQ AAAA 3ijeYFE ~ Zup 9nhRYBuHm1KS2tVqBG...,
2029,3802,560092063048679424,2015-01-27 10:00:00,no media,False,False,no media,0.0,0.0,0.0,Some very useful videos from ConEd on saving...,/julieczka/status/560092063048679424,23007537.0,julieczka,Some very useful video from ConEd on saving mo...,video money heating cost winter power outage b...,ConEd videos/?bcpid= AQ AAAA 3ijeYFE ~ Zup 9nh...,


In [116]:
df_2012_2017[df_2012_2017.is_reply == True]

,index,ID,datetime,has_media,is_reply,is_retweet,medias,nbr_favorite,nbr_reply,nbr_retweet,text,url,user_id,usernameTweet,lemmed_tweet,nouns,proper_nouns,geopolitical_ents
463,3785,158571204138115072,2012-01-15 10:00:00,no media,True,False,no media,0.0,1.0,0.0,@ That_MF there was a building Change this mo...,/MRDEZFAMOUS/status/158571204138115072,1.530571e+08,MRDEZFAMOUS,@ That_MF there wa a building Change this morn...,building morning clock,Change Power Outage Sliver Building Coned,
464,4339,158606705880604672,2012-01-15 12:00:00,no media,True,False,no media,0.0,1.0,0.0,@WutNfNBrat Yes I work at the Sliver Building ...,/MRDEZFAMOUS/status/158606705880604672,1.530571e+08,MRDEZFAMOUS,@WutNfNBrat Yes I work at the Sliver Building ...,outage clock fix,@WutNfNBrat Sliver Building Today Black Buildi...,Power
466,2350,162891703425048576,2012-01-27 08:00:00,no media,True,False,no media,0.0,1.0,0.0,@ridinchillwaves There's a power outage in...,/News12EP/status/162891703425048576,3.611640e+08,News12EP,@ridinchillwaves There's a power outage in par...,power outage part Power,Bronx ConEd,Bronx
498,289,220554948377706497,2012-07-04 12:00:00,no media,True,False,no media,0.0,0.0,0.0,@ NewYorkPost Heard Deutsche Welle rep on C-S...,/Michaelmrw/status/220554948377706497,1.006251e+08,Michaelmrw,@ NewYorkPost Heard Deutsche Welle rep on C-Sp...,rep Span people time,Heard Deutsche Welle C Power Outage ConEd Germ...,US
514,560,223059133283303425,2012-07-11 10:00:00,no media,True,False,no media,0.0,0.0,0.0,@ Newyorkist why can't we sue #ConEd for po...,/solomon_456/status/223059133283303425,2.045410e+08,solomon_456,@ Newyorkist why can't we sue #ConEd for power...,ConEd power outage,Newyorkist,
515,2145,223059282327896064,2012-07-11 10:00:00,no media,True,False,no media,0.0,0.0,0.0,@ DantePosh why can't we sue #ConEd for pow...,/solomon_456/status/223059282327896064,2.045410e+08,solomon_456,@ DantePosh why can't we sue #ConEd for power ...,ConEd power outage,DantePosh,
519,385,224863471672623106,2012-07-16 09:00:00,no media,True,False,no media,0.0,0.0,0.0,@ JuneMo41 @ConEdison How have you suffered?...,/Teldar_and_Co/status/224863471672623106,6.252421e+08,Teldar_and_Co,@ JuneMo 41 @ConEdison How have you suffered ?...,power outage yesterday union job,JuneMo @ConEdison Staten Island,Staten Island
537,2858,225928276105904128,2012-07-19 08:00:00,no media,True,False,no media,0.0,0.0,0.0,@ claudiaUME actually it's a power outage...,/AbbieLicious07/status/225928276105904128,1.950049e+08,AbbieLicious07,@ claudiaUME actually it's a power outage in t...,power outage building ppl issue side morning,,
553,3922,244555615240871936,2012-09-08 17:00:00,no media,True,False,no media,0.0,0.0,0.0,@ wunderground keep an eye on @stormpocalypse...,/ckundo/status/244555615240871936,1.478986e+07,ckundo,@ wunderground keep an eye on @stormpocalypse ...,eye power outage area,ConEd NYC,
569,2032,261955915803136002,2012-10-26 18:00:00,no media,True,False,no media,0.0,0.0,0.0,RT @MikeBloomberg NYers should be prepared fo...,/oaaigbe/status/261955915803136002,1.486504e+08,oaaigbe,RT @MikeBloomberg NYers should be prepared for...,possibility power outage #,RT @MikeBloomberg NYers @ConEdison Sandy,
